In [1]:
#load text processing NLP classes and packages
import pandas as pd
import numpy as np
from string import punctuation
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer
import pickle
from nltk.stem import PorterStemmer
import os
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer #loading tfidf vector
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, Bidirectional, GRU
from keras.models import Sequential, load_model, Model
import pickle
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import GridSearchCV

Using TensorFlow backend.
C:\Users\S.Gyanjoth Singh\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\S.Gyanjoth Singh\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\S.Gyanjoth Singh\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\S.Gyanjoth Singh\Anaconda3\lib\site-pa

In [35]:
#define global variables for text processing such as lemmatization and stopwords
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()

In [36]:
#define function to clean text by removing stop words and other special symbols
def cleanText(doc):
    tokens = doc.split()
    table = str.maketrans('', '', punctuation)
    tokens = [w.translate(table) for w in tokens]
    tokens = [word for word in tokens if word.isalpha()]
    tokens = [w for w in tokens if not w in stop_words]
    tokens = [word for word in tokens if len(word) > 1]
    tokens = [ps.stem(token) for token in tokens]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    tokens = ' '.join(tokens)
    return tokens

In [41]:
#load spam dataset
dataset = pd.read_csv("Dataset/SPAM.csv")
dataset

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [42]:
#read all messages and clean them
data = dataset.values
X = []
Y = []
for i in range(len(data)):#loop all messages
    msg = data[i,1]
    msg = msg.strip("\n").strip().lower()#read msg
    label = data[i,0]#read labels
    if label == "ham":
        label = 0
    else:
        label = 1
    msg = cleanText(msg)#clean message
    X.append(msg)
    Y.append(label)
X = np.asarray(X)
Y = np.asarray(Y)
print("Dataset Cleaning Completed")

Dataset Cleaning Completed


In [43]:
#creating fixed size vector model from messages
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words, max_features=300)
X = tfidf_vectorizer.fit_transform(X).toarray()
data = pd.DataFrame(X, columns=tfidf_vectorizer.get_feature_names())
data

,account,actual,aight,alreadi,also,alway,amp,anoth,answer,anyth,...,worri,would,xxx,ya,ye,yeah,year,yet,yo,yup
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.376148,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5568,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5569,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5570,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
#normalizing features
sc = StandardScaler()
X = sc.fit_transform(X)
print("Normalized Features = "+str(X))
Y = to_categorical(Y)
X = np.reshape(X, (X.shape[0], 30, 10))

Normalized Features = [[-0.08020853 -0.07303585 -0.07580714 ... -0.08855605 -0.07462695
  -0.08389389]
 [-0.08020853 -0.07303585 -0.07580714 ... -0.08855605 -0.07462695
  -0.08389389]
 [-0.08020853 -0.07303585 -0.07580714 ... -0.08855605 -0.07462695
  -0.08389389]
 ...
 [-0.08020853 -0.07303585 -0.07580714 ... -0.08855605 -0.07462695
  -0.08389389]
 [-0.08020853 -0.07303585 -0.07580714 ... -0.08855605 -0.07462695
  -0.08389389]
 [-0.08020853 -0.07303585 -0.07580714 ... -0.08855605 -0.07462695
  -0.08389389]]


In [45]:
#split dataset to train & test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
print("Training 80% Records = "+str(X_train.shape[0]))
print("Testing 20% Records = "+str(X_test.shape[0]))

Training 80% Records = 4457
Testing 20% Records = 1115


In [46]:
#define LSTM algorithm
lstm_model = Sequential()#defining deep learning sequential object
#adding LSTM layer with 100 filters to filter given input X train data to select relevant features
lstm_model.add(LSTM(64,input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
lstm_model.add(Dropout(0.5))
lstm_model.add(Bidirectional(GRU(32, return_sequences=True)))
#adding dropout layer to remove irrelevant features
lstm_model.add(Dropout(0.5))
lstm_model.add(Bidirectional(GRU(32)))
lstm_model.add(Dropout(0.5))
#adding another layer
lstm_model.add(Dense(100, activation='relu'))
#defining output layer for prediction
lstm_model.add(Dense(y_train.shape[1], activation='softmax'))
#compile the model
lstm_model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
lstm_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 30, 64)            19200     
_________________________________________________________________
dropout_4 (Dropout)          (None, 30, 64)            0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 30, 64)            18624     
_________________________________________________________________
dropout_5 (Dropout)          (None, 30, 64)            0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, 64)                18624     
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)              

In [47]:
#tuning LSTM using hyper parameters
param_grid = {
    'epochs': [3, 5, 10],
    'batch_size': [16, 32, 64],    
}
best_model = None
best_score = 0
best_param = None
#training LSTM using tuning parameters
result = [dict(zip(param_grid.keys(), v)) for v in zip(*param_grid.values())]
for i in range(len(result)):
    epoch = result[i]['epochs']
    batch_size  = result[i]['batch_size']
    model_check_point = ModelCheckpoint(filepath='model/lstm_weights.hdf5', verbose = 1, save_best_only = True)
    lstm_model.fit(X_train, y_train, batch_size = batch_size, epochs = epoch, validation_data=(X_test, y_test), callbacks=[model_check_point], verbose=1)
    loss, accuracy = lstm_model.evaluate(X_test, y_test, verbose = 0)
    if accuracy > best_score:
        best_score = accuracy
        best_model = lstm_model
        best_param = "Epochs: "+str(epoch)+" Batch Size: "+str(batch_size)

Train on 4457 samples, validate on 1115 samples
Epoch 1/3
4457/4457 [==============================] - 56s 13ms/step - loss: 0.2846 - accuracy: 0.8932 - val_loss: 0.2313 - val_accuracy: 0.9184

Epoch 00001: val_loss improved from inf to 0.23133, saving model to model/lstm_weights.hdf5
Epoch 2/3
4457/4457 [==============================] - 58s 13ms/step - loss: 0.2258 - accuracy: 0.9172 - val_loss: 0.2051 - val_accuracy: 0.9354

Epoch 00002: val_loss improved from 0.23133 to 0.20510, saving model to model/lstm_weights.hdf5
Epoch 3/3
4457/4457 [==============================] - 59s 13ms/step - loss: 0.1960 - accuracy: 0.9309 - val_loss: 0.1790 - val_accuracy: 0.9471

Epoch 00003: val_loss improved from 0.20510 to 0.17903, saving model to model/lstm_weights.hdf5
Train on 4457 samples, validate on 1115 samples
Epoch 1/5
4457/4457 [==============================] - 37s 8ms/step - loss: 0.1696 - accuracy: 0.9394 - val_loss: 0.1658 - val_accuracy: 0.9444

Epoch 00001: val_loss improved from i

In [48]:
print("Best Score: "+str(best_score))
print("Best Tuning Params: "+str(best_param))

Best Score: 0.9524663686752319
Best Tuning Params: Epochs: 10 Batch Size: 64


In [50]:
#evaluating best model performance
predict = best_model.predict(X_test)#performing prediction on test data using best model
predict = np.argmax(predict, axis=1)
y_test1 = np.argmax(y_test, axis=1)
a = accuracy_score(y_test1,predict)*100 #calculate accuracy and other metrics using original labels and predicted labels
p = precision_score(y_test1, predict,average='macro') * 100
r = recall_score(y_test1, predict,average='macro') * 100
f = f1_score(y_test1, predict,average='macro') * 100
print("Best Model Performance Measure")
print("Accuracy: "+str(a))
print("Precision: "+str(p))
print("Recall: "+str(r))
print("FSCORE: "+str(f))

Best Model Performance Measure
Accuracy: 95.24663677130046
Precision: 91.82831776594726
Recall: 86.67269906928645
FSCORE: 89.00813764240874
